# АОЕЯ Hometask №1
### Феоктистова Эмма
### 3 курс ФиКЛ

#### __*Этап 1:*__

Сначала скачаем дату (возьмем отзывы о бирже фриланса FL.ru с сайта otzyvmarketing.ru):

In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
def get_comments(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html')
    
    half_matches = soup.find_all('div', class_='reviews_wrapper')
    all_matches.extend(half_matches)

In [3]:
urls = ['https://otzyvmarketing.ru/fl-ru/', 'https://otzyvmarketing.ru/fl-ru/?page=2', 'https://otzyvmarketing.ru/fl-ru/?page=3', 'https://otzyvmarketing.ru/fl-ru/?page=4']
all_matches = []

for url in urls:
    get_comments(url)

В сумме получилось 166 комментариев с различной оценкой пользователей (от 1й звезды до 5ти). 

In [4]:
len(all_matches)

166

Разделим полученные комментарии сразу на положительные и отрицательные (в качестве положительных будем считать отзывы с 4мя и 5ю звездами, отрицательных - 1-3 звезды):

In [5]:
good = []
bad = []
for i in all_matches:
    try:
        stars = i.find('div', {'class': 'stages_inner'})['style']
    except:
        pass
    
    if stars == 'width: 100%;' or stars == 'width: 80%;':
        good.append(i)
    elif stars == 'width: 20%;' or stars == 'width: 40%;' or stars == 'width: 60%;':
        bad.append(i)

In [6]:
print(len(good))
print(len(bad))

58
108


И вытащим из нашего html кода именно тексты отзывов:

In [7]:
good_comments = []
bad_comments = []
for g in good:
    try:
        good_text = g.find('p', {'itemprop': "description"}).text
        good_text = good_text.replace('\xa0', ' ').replace('\r', ' ').replace('\n', ' ')
        good_comments.append(good_text)
    except:
        pass

for b in bad:
    try:
        bad_text = b.find('p', {'itemprop': "description"}).text
        bad_text = bad_text.replace('\xa0', ' ').replace('\r', ' ').replace('\n', ' ')
        bad_comments.append(bad_text)
    except:
        pass

In [8]:
testing_good = good_comments[0:10]
testing_bad = bad_comments[0:10]
good_comments = good_comments[10:]
bad_comments = bad_comments[10:]

#### __*Этап 2:*__

Токенизируем слова, предварительно приведя их в нижний регистр. Отсечем пунктуационные знаки и найдем начальную форму каждого слова. Все полученные данные также разделим на хорошие и плохие. 

In [9]:
import nltk
import pymorphy2
from collections import Counter

morph = pymorphy2.MorphAnalyzer()

good_prepared = []
bad_prepared = []

for i in good_comments:
    good_words = nltk.word_tokenize(i.lower())
    
    for word in good_words:
        if word.isalpha():
            m = morph.parse(word)[0]
            good_prepared.append(m.normal_form)

for j in bad_comments:
    bad_words = nltk.word_tokenize(j.lower())

    for word in bad_words:
        if word.isalpha():
            m = morph.parse(word)[0]
            bad_prepared.append(m.normal_form)

#### __*Этап 3:*__

Составим множества слов, которые встречаются только в положительных комментариях и только в отрицательных.  Посчитаем их количество с помощью модуля Counter() и выведем самые частотные слова. 

In [10]:
good_counter = Counter()
for word in good_prepared:
    if word not in bad_prepared:
        good_counter[word] += 1

good_counter.most_common(30)


[('доступный', 7),
 ('разработка', 7),
 ('копирайтинг', 6),
 ('уверить', 5),
 ('недостаток', 5),
 ('достоинство', 4),
 ('пришлый', 4),
 ('скидка', 4),
 ('профессиональный', 4),
 ('направление', 4),
 ('пополняться', 4),
 ('выступать', 4),
 ('счёт', 3),
 ('наличие', 3),
 ('решаться', 3),
 ('необходимость', 3),
 ('урок', 3),
 ('довольный', 3),
 ('тщательно', 3),
 ('доверять', 3),
 ('радовать', 3),
 ('достойный', 3),
 ('технический', 3),
 ('оперативно', 3),
 ('маркетинговый', 3),
 ('агентство', 3),
 ('анимация', 3),
 ('моделирование', 3),
 ('документ', 3),
 ('блог', 3)]

In [11]:
bad_counter = Counter()
for word in bad_prepared:
    if word not in good_prepared:
        bad_counter[word] += 1
        
bad_counter.most_common(30)

[('переписка', 15),
 ('публикация', 14),
 ('вернуть', 14),
 ('сообщение', 13),
 ('пост', 13),
 ('никак', 12),
 ('ограничение', 11),
 ('ваш', 11),
 ('должный', 11),
 ('либо', 10),
 ('создавать', 10),
 ('фейковыя', 10),
 ('смотреть', 10),
 ('заблокировать', 10),
 ('конкурс', 10),
 ('уходить', 9),
 ('читать', 9),
 ('отправить', 9),
 ('нововведение', 9),
 ('письмо', 9),
 ('надо', 9),
 ('куча', 9),
 ('страница', 9),
 ('реальный', 9),
 ('опубликовать', 8),
 ('бс', 8),
 ('обращение', 7),
 ('просить', 7),
 ('ответить', 7),
 ('продолжать', 7)]

#### __*Этап 4:*__

Напишем функции, которые определяют положительный ли комментарий дан ей, либо отрицательный. Для этого аналогично подготавливаем текст проверяемого комментария, и проверяем, сколько слов из наших множеств встречаются в данном комментарии. 

In [12]:
from sklearn.metrics import accuracy_score

def good_checking(comment):
    is_good = 0
    words = nltk.word_tokenize(comment.lower())
    
    to_check_good = []
    for word in words:
        if word.isalpha():
            m = morph.parse(word)[0]
            to_check_good.append(m.normal_form)
    
    for word in to_check_good:
        if word in good_counter:
            is_good += 1
  
    return is_good
    
def bad_checking(comment):
    is_bad = 0
    words = nltk.word_tokenize(comment.lower())
    
    to_check_bad = []
    for word in words:
        if word.isalpha():
            m = morph.parse(word)[0]
            to_check_bad.append(m.normal_form)
            
    for word in to_check_bad:
        if word in bad_counter:
            is_bad += 1

    return is_bad

Возьмем наши тестовые комментарии, и проверим модель. также посчитаем accuracy нашей модели. 

In [13]:
results1 = []

for i in testing_good:
    g_number = good_checking(i)
    b_number = bad_checking(i)
    
    if g_number > b_number:
        results1.append(i)
        print('Комментарий положительный.')
    else:
        results1.append('-')
        

a1 = accuracy_score(results1, testing_good)
print('\n' + 'Точность предсказания: ' + f'{a1:.4f}')

Комментарий положительный.
Комментарий положительный.
Комментарий положительный.

Точность предсказания: 0.3000


In [14]:
results2 = []

for j in testing_bad:
    g_number = good_checking(j)
    b_number = bad_checking(j)
    
    if g_number < b_number:
        results2.append(j)
        print('Комментарий отрицательный.')
    else:
        results2.append('-')
        

a2 = accuracy_score(results2, testing_bad)
print('\n' + 'Точность предсказания: ' + f'{a2:.4f}')

Комментарий отрицательный.
Комментарий отрицательный.
Комментарий отрицательный.
Комментарий отрицательный.
Комментарий отрицательный.
Комментарий отрицательный.
Комментарий отрицательный.
Комментарий отрицательный.
Комментарий отрицательный.

Точность предсказания: 0.9000


К сожалению, из-за небольшого кол-ва данных точность разнится (что можно пронаблюдать в случае с положительными комментариями). Также из-за наличия более менее нейтральных комментариев с 3-мя звездами множества пол./отр. слов не столь показательны и разняться между собой. 

### __*Предложения и пожелания:*__

**Совет по оптимизации №1:**
Например, собирать данные (отзывы/комментарии) с нескольких сайтов автоматически. Имеется в виду та ситуация, когда нужны отзывы о табуретках (пример), программа заходит на сайты с отзывами, забивает в поиске нужный товар и собирает данные. Тем самым выборка получится больше = множества корректнее = обученная модель точнее. 

**Совет по оптимизации №2:**
Включить в параметры положительности/отрицательности отзыва не только слова, но и пунктуацию (типа: _табуретка огонь!!!!!_), а также смайлики, которые аналогично имеют эмоциональный окрас. В код в данном случае должны быть добавлены шаблоны с различными применениями пунктуации как отражения эмоций пользователя (еще пример - скобочки как показатель радости/грусти ("_табуретка шатается((_"). Также определители смайликов (например по id смайла определять, что он означает)).
